In [ ]:
# Quantize

not_supported_parameters = {
    'weights': {
        "bits": int,
        "mode": ["symmetric", "asymetric"],
        "perchannel": [True, False], # granularity
        'level_low': None,
        'level_high': None,
        "range_estimator": {
            "preset": [None, "default", "quantile"],
            "min": {
                'type': ['max', 'min', 'abs_max', 'quantile', 'abs_quantile'],
                'outlier_prob': [None, float],
                'granularity': ['pertensor', 'perchannel'],
                'clipping_value': [None, float]
            }, 
            "max": {
                'type': ['max', 'min', 'abs_max', 'quantile', 'abs_quantile'],
                'outlier_prob': [None, float],
                'granularity': ['pertensor', 'perchannel'],
                'clipping_value': [None, float]
            }
        }
    },
    'activations': {
        "bits": int,
        "mode": ["symmetric", "asymetric"],
        "perchannel": [True, False], # granularity
        'level_low': None,
        'level_high': None,
        "range_estimator": {
            "preset": [None, "default", "quantile"],
            "min": {
                'type': ['max', 'min', 'abs_max', 'quantile', 'abs_quantile', 'mean'],
                'aggregator': ['mean', 'max', 'min', 'median', 'mean_no_outliers', 'median_no_outliers', 'hl_estimator'], 42 варианта
                'outlier_prob': [None, float],
                'granularity': ['pertensor', 'perchannel'],
                'clipping_value': [None, float]
            }, 
            "max": {
                'type': ['max', 'min', 'abs_max', 'quantile', 'abs_quantile', 'mean'],
                'aggregator': ['mean', 'max', 'min', 'median', 'mean_no_outliers', 'median_no_outliers', 'hl_estimator'],
                'outlier_prob': [None, float],
                'granularity': ['pertensor', 'perchannel'],
                'clipping_value': [None, float]
            }
        }
    },
    "range_estimator": {
        "preset": [None, "default", "quantile"],
        "min": {
            'type': ['max', 'min', 'abs_max', 'quantile', 'abs_quantile'],
            'outlier_prob': [None, float],
            'granularity': ['pertensor', 'perchannel'],
            'clipping_value': [None, float]
        }, 
        "max": {
            'type': ['max', 'min', 'abs_max', 'quantile', 'abs_quantile'],
            'outlier_prob': [None, float],
            'granularity': ['pertensor', 'perchannel'],
            'clipping_value': [None, float]
        }
    },
    'apply_for_all_nodes':[True, False], # BiasCorrection
    'threshold': [None, float], # BiasCorrection
    'saturation_fix': [True, False],
    'inplace_statistics': [True, False]
}

In [ ]:
'overflow_fix': [
    'enable',
    'disable',
    'first_layer_only'
]

QConfig: ['signed', 'target_scopes', 'logarithm_scale', 'unified_scale_ops'=List[str]]

In [ ]:
{
    "type": "percentile",
    "num_init_samples": 10,
    "params": {
        "min_percentile": "0.1",
        "max_percentile": "99.9"
    },
    "target_quantizer_group": "activations",
    "target_scopes": [
        "TwoConvTestModel/Sequential[features]/Sequential[1]/NNCFConv2d[0]/conv2d_0|OUTPUT"]
}

In [ ]:
'type':

"mixed_min_max": "Minimum quantizer range initialized using minima of per-channel minima of "
                    "the tensor to be quantized, maximum quantizer range initialized using "
                    "maxima of per-channel maxima of the tensor to be quantized. Offline.",
"min_max": "Minimum quantizer range initialized using global minimum of values in "
            "the tensor to be quantized, maximum quantizer range initialized using global maxima of the same"
            "values. Online.",
"mean_min_max": "Minimum quantizer range initialized using averages (across every single initialization sample) "
                "of minima of values in "
                "the tensor to be quantized, maximum quantizer range initialized using maxima respectively. "
                "Offline.",
"threesigma": "Quantizer minimum and maximum ranges set to be equal to +- 3 median absolute deviation from the "
                "median of the observed values in the tensor to be quantized. Offline.",
"percentile": "Quantizer minimum and maximum ranges set to be equal to specified percentiles of the the observed "
                "values (across the entire initialization sample set) in the tensor to be quantized. Offline.",
"mean_percentile": "Quantizer minimum and maximum ranges set to be equal to averaged (across every single "
                    "initialization sample) specified percentiles of the the observed values in the tensor to be "
                    "quantized. Offline."
                    
                    
                    
"min_percentile":
    with_attributes(NUMBER,
                    description="For 'percentile' and 'mean_percentile' types - specify the percentile "
                                "of input value histograms to be set as the initial "
                                "value for the quantizer input minimum.",
                    default=MIN_PERCENTILE),
"max_percentile":
    with_attributes(NUMBER,
                    description="For 'percentile' and 'mean_percentile' types - specify the percentile "
                                "of input value histograms to be set as the initial "
                                "value for the quantizer input maximum.",
                    default=MAX_PERCENTILE),

- Нужна ли нам гибкость в range_estimator как в POT
  - IMHO: на уровне quantize функции такая гибкость не нужна, достаточно пресетов.

- Нужен ли global range_estimator?
  - IMHO: выглядит так, что range_estimator parameters для activations and weights используются в github repos 

- target_scope
  - IMHO: Не понятно кто это использует... предлагаю припарковать вопрос и рассмотреть альтернотивные варианты

- signed, logarithm_scale
  - IMHO: параметры применяются для QAT, на данном этапе предлагаю исключить

- unified_scale_ops
  - IMHO: открытый вопрос? Нужны сценарии использования

# Naive approach

In [ ]:
from dataclasses import InitVar
from typing import Optional

class WeightsRangeEstimatorType(Enum):
    MAX = 'max'
    MIN = 'min'
    ABS_MAX = 'abs_max'
    QUANTILE = 'quantile'
    ABS_QUANTILE = 'abs_quantile'


class ActivationRangeEstimatorType(Enum):
    MAX = 'max'
    MIN = 'min'
    ABS_MAX = 'abs_max'
    QUANTILE = 'quantile'
    ABS_QUANTILE = 'abs_quantile'
    MEAN = 'mean'


class Aggregator(Enum):
    MEAN = 'mean'
    MAX = 'max'
    MIN = 'min'
    MEDIAN ='median'
    MEAN_NO_OUTLIERS = 'mean_no_outliers'
    MEDIAN_NO_OUTLIERS = 'median_no_outliers'


class OverflowFix(Enum):
    ENABLE = 'enable'
    DISABLE = 'disable'
    FIRST_LAYER_ONLY = 'first_layer_only'
    

@dataclass
class AdvancedQuantizationParameters:
    # activation
    activations_num_bits: InitVar[Optional[int]] = None
    activations_qunatization_mode: InitVar[Optional[QuantizationMode]] = None
    activations_per_channel: InitVar[Optional[bool]] = None 
    activations_range_estimator_min_type: InitVar[Optional[ActivationRangeEstimatorType]] = None
    activations_range_estimator_min_agragator: InitVar[Optional[Aggregator]] = None
    activations_range_estimator_min_clippong_value: InitVar[Optional[float]] = None
    activations_range_estimator_min_outlier_prob: InitVar[Optional[float]] = None
    activations_range_estimator_max_type: InitVar[Optional[ActivationRangeEstimatorType]] = None
    activations_range_estimator_max_agragator: InitVar[Optional[Aggregator]] = None
    activations_range_estimator_max_clippong_value: InitVar[Optional[float]] = None
    activations_range_estimator_max_outlier_prob: InitVar[Optional[float]] = None
    
    # weights
    weights_num_bits: InitVar[Optional[int]] = None
    weights_qunatization_mode: InitVar[Optional[QuantizationMode]] = None
    weights_per_channel: InitVar[Optional[bool]] = None
    weights_half_range: InitVar[Optional[bool]] = None
    weights_narrow_range: InitVar[Optional[bool]] = None
    weights_range_estimator_min_type: InitVar[Optional[WeightsRangeEstimatorType]] = None
    weights_range_estimator_min_clippong_value: InitVar[Optional[float]] = None
    weights_range_estimator_min_outlier_prob: InitVar[Optional[float]] = None
    weights_range_estimator_max_type: InitVar[Optional[WeightsRangeEstimatorType]] = None
    weights_range_estimator_max_clippong_value: InitVar[Optional[float]] = None
    weights_range_estimator_max_outlier_prob: InitVar[Optional[float]] = None
       
    # common    
    bias_correction_apply_for_all_nodes: InitVar[Optional[bool]] = None # BiasCorrection
    bias_correction_threshold: InitVar[Optional[float]] = None # BiasCorrection
    overflow_fix: InitVar[Optional[OverflowFix]] = None # ['enable', 'disable', 'first_layer_only'] thre is saturation_fix in POT.
    inplace_statistics: InitVar[Optional[bool]] = None
    
    #backend specific parameters
    backend_parameters: InitVar[Optional[Dict]] = None # backend specific paramaters


def quantize(
    model: TModel,
    calibration_dataset: Dataset,
    preset: QuantizationPreset = QuantizationPreset.PERFORMANCE,
    target_device: TargetDevice = TargetDevice.ANY,
    subset_size: int = 300,
    fast_bias_correction: bool = True,
    model_type: Optional[ModelType] = None,
    ignored_scope: Optional[IgnoredScope] = None,
    advanced_parameters: Optional[AdvancedQuantizationParameters()] = None) -> TModel:
pass

# how to use

advanced_parameters = AdvancedQuantizationParameters(
    activation_num_bits=8,
    activation_range_estimator_min_type="min",
    activation_range_estimator_min_agragator="mean",
    activation_range_estimator_max_type="max",
    activation_range_estimator_max_agragator="mean",
    weights_num_bits=8,
    weights_per_channel=True,
    bias_correction_apply_for_all_nodes = True,
    backend_parameters={
        "num_infer_request": 2
    }
)

qmodel = quantize(model=model, calibration_dataset=dataset, advanced_parameters=advanced_parameters)


# Naive Dict

In [ ]:
from dataclasses import InitVar, dataclass
from enum import Enum
from typing import Any, Dict, Optional


class WeightsStatisticFunction(Enum):
    MAX = 'max'
    MIN = 'min'
    ABS_MAX = 'abs_max'
    QUANTILE = 'quantile'
    ABS_QUANTILE = 'abs_quantile'


class ActivationStatisticFunction(Enum):
    MAX = 'max'
    MIN = 'min'
    ABS_MAX = 'abs_max'
    QUANTILE = 'quantile'
    ABS_QUANTILE = 'abs_quantile'
    MEAN = 'mean'


class AggregatorFunction(Enum):
    MEAN = 'mean'
    MAX = 'max'
    MIN = 'min'
    MEDIAN ='median'
    MEAN_NO_OUTLIERS = 'mean_no_outliers'
    MEDIAN_NO_OUTLIERS = 'median_no_outliers'


class OverflowFix(Enum):
    ENABLE = 'enable'
    DISABLE = 'disable'
    FIRST_LAYER_ONLY = 'first_layer_only'


def get_advanced_quantizatization_parameters(preset: QuantizationPreset):
    if preset == QuantizationPreset.PERFORMANCE:
        retrun {
            'weights': {
                "num_bits": 8,
                "mode": QuantizationMode.SYMMETRIC,
                "perchannel": True,
                "range_estimator": {
                    "min": {
                        'type': WeightsStatisticFunction.MIN,
                        'outlier_prob': None,
                        'clipping_value': None
                    }, 
                    "max": {
                        'type': WeightsStatisticFunction.MAX,
                        'outlier_prob': None,
                        'clipping_value': None
                    }
                }
            },
            'activations': {
                "num_bits": 8,
                "mode": QuantizationMode.SYMMETRIC,
                "perchannel": None,
                "range_estimator": {
                    "min": {
                        'type': ActivationStatisticFunction.MIN,
                        'aggregator': AggregatorFunction.MEAN,
                        'outlier_prob': None,
                        'clipping_value': None
                    }, 
                    "max": {
                        'type': ActivationStatisticFunction.MAX,
                        'aggregator': AggregatorFunction.MEAN,
                        'outlier_prob': None,
                        'clipping_value': None
                    }
                }
            },
            'apply_bias_correction_for_all_nodes': True,
            'threshold': None,
            'overflow_fix': OverflowFix.FIRST_LAYER_ONLY,
            'inplace_statistics': True,
            'backend_parameters': {}
        }


def quantize(
    model: TModel,
    calibration_dataset: Dataset,
    preset: QuantizationPreset = QuantizationPreset.PERFORMANCE,
    target_device: TargetDevice = TargetDevice.ANY,
    subset_size: int = 300,
    fast_bias_correction: bool = True,
    model_type: Optional[ModelType] = None,
    ignored_scope: Optional[IgnoredScope] = None,
    advanced_parameters: Optional[Dict] = None) -> TModel:
pass

# how to use

advanced_parameters = get_advanced_quantizatization_parameters(QuantizationPreset.PERFORMANCE)
advanced_parameters['activations']['num_bits'] = 8
advanced_parameters['activations']['range_estimator']['min']['type'] = ActivationStatisticFunction.MIN
advanced_parameters['activations']['range_estimator']['min']['agragator'] = AggregatorFunction.MEAN
advanced_parameters['activations']['range_estimator']['max']['type'] = ActivationStatisticFunction.MAX
advanced_parameters['activations']['range_estimator']['max']['agragator'] = AggregatorFunction.MEAN
advanced_parameters['weights']['num_bits'] = 8
advanced_parameters['weights']['per_channel'] = False
advanced_parameters['bias_correction_apply_for_all_nodes'] = True
advanced_parameters['backend_parameters'] = {"num_infer_request": 2}

# OR

advanced_parameters.update(
    {
        'weights': {
                "num_bits": 8,
                "perchannel": False,
        },
        'activations': {
            "num_bits": 8,
            "range_estimator": {
                "min": {
                    'type': ActivationStatisticFunction.MIN,
                    'aggregator': AggregatorFunction.MEAN,
                }, 
                "max": {
                    'type': ActivationStatisticFunction.MAX,
                    'aggregator': AggregatorFunction.MEAN,
                }
            }
        },
        'apply_bias_correction_for_all_nodes': True,
        'backend_parameters': {'num_infer_request': 2}
    }
)

qmodel = quantize(model, calibration_dataset=dataset, advanced_parameters=advanced_parameters)


# Compatoble with POT config

In [ ]:
from dataclasses import dataclass
from enum import Enum
from typing import Any, Dict, Optional

from nncf.common.quantization.structs import QuantizationMode


class WeightsStatisticFunction(Enum):
    MAX = 'max'
    MIN = 'min'
    ABS_MAX = 'abs_max'
    QUANTILE = 'quantile'
    ABS_QUANTILE = 'abs_quantile'


class ActivationsStatisticFunction(Enum):
    MAX = 'max'
    MIN = 'min'
    ABS_MAX = 'abs_max'
    QUANTILE = 'quantile'
    ABS_QUANTILE = 'abs_quantile'
    MEAN = 'mean'


class AggregationFunction(Enum):
    MEAN = 'mean'
    MAX = 'max'
    MIN = 'min'
    MEDIAN = 'median'
    MEAN_NO_OUTLIERS = 'mean_no_outliers'
    MEDIAN_NO_OUTLIERS = 'median_no_outliers'


class OverflowFix(Enum):
    ENABLE = 'enable'
    DISABLE = 'disable'
    FIRST_LAYER_ONLY = 'first_layer_only'


@dataclass
class WeightsEstimatorParameters():
    statistic_function:Optional[WeightsStatisticFunction] = None 
    statistic_function_params: Optional[Dict] = None
    clipping_value: Optional[float] = None


@dataclass
class ActivationsEstimatorParameters():
    statistic_function: Optional[ActivationsStatisticFunction] = None 
    statistic_function_params: Optional[Dict] = None
    aggregation_function: Optional[AggregationFunction] = None
    clipping_value: Optional[float] = None


@dataclass
class ActivationsRangeEstimatorParameters:
    min: Optional[ActivationsEstimatorParameters] = None
    max: Optional[ActivationsEstimatorParameters] = None


@dataclass
class WeightsRangeEstimatorParameters:
    min: Optional[WeightsEstimatorParameters] = None
    max: Optional[WeightsEstimatorParameters] = None


@dataclass
class ActivationsQuantizationParameters:
    num_bits: Optional[int] = None
    mode: Optional[QuantizationMode] = None
    per_channel: Optional[bool] = None # granularity
    range_estimator: Optional[ActivationsRangeEstimatorParameters] = None


@dataclass
class WeightsQuantizationParameters:
    num_bits: Optional[int] = None
    mode: Optional[QuantizationMode] = None
    per_channel: Optional[bool] = None
    half_range: Optional[bool] = None
    narrow_range: Optional[bool] = None
    range_estimator: Optional[WeightsRangeEstimatorParameters] = None


@dataclass
class AdvancedQuantizationParameters:
    # quantization configurations for activations and weights
    activations: Optional[ActivationsQuantizationParameters] = None
    weights: Optional[WeightsQuantizationParameters] = None
    #activations_quantization_params: Optional[ActivationsQuantizationParameters] = None
    #weights_quantization_params: Optional[WeightsQuantizationParameters] = None
       
    # BiasCorrection algorithm parameters
    apply_for_all_nodes = Optional[bool] = None
    threshold = Optional[float] = None
    
    # general parameters
    overflow_fix: Optional[OverflowFix] = None
    inplace_statistics: Optional[bool] = None
    
    # backend specific parameters
    backend_params: Optional[Dict[str, Any]] = None
     

def quantize(
    model: TModel,
    calibration_dataset: Dataset,
    preset: QuantizationPreset = QuantizationPreset.PERFORMANCE,
    target_device: TargetDevice = TargetDevice.ANY,
    subset_size: int = 300,
    fast_bias_correction: bool = True,
    model_type: Optional[ModelType] = None,
    ignored_scope: Optional[IgnoredScope] = None,
    advanced_parameters: Optional[AdvancedQuantizationParameters()] = None) -> TModel:
pass

# how to use

advanced_parameters = AdvancedQuantizationParameters(
    activations=ActivationsQuantizationParameters(
        num_bits=8, 
        range_estimator=ActivationsRangeEstimatorParameters(
            min=ActivationsEstimatorParameters(
                statistic_function=ActivationStatisticFunction.MIN,
                aggregator=AggregatorFunction.MEAN),
            max=ActivationsEstimatorParameters(
                statistic_function=ActivationStatisticFunction.MAX,
                aggregator=AggregatorFunction.MEAN))),
    weights=WeightsQuantizationParameters(
        num_bits=8,
        per_channel=True),
    apply_for_all_nodes = True,
    backend_params={
        "num_infer_request": 2
    }
)

qmodel = quantize(model, calibration_dataset=dataset, advanced_parameters=advanced_parameters)

# Possible improvements

In [ ]:
from dataclasses import dataclass
from enum import Enum
from typing import Any, Dict, Optional

from nncf.common.quantization.structs import QuantizationMode


class WeightsStatisticFunction(Enum):
    MAX = 'max'
    MIN = 'min'
    ABS_MAX = 'abs_max'
    QUANTILE = 'quantile'
    ABS_QUANTILE = 'abs_quantile'


class ActivationsStatisticFunction(Enum):
    MAX = 'max'
    MIN = 'min'
    ABS_MAX = 'abs_max'
    QUANTILE = 'quantile'
    ABS_QUANTILE = 'abs_quantile'
    MEAN = 'mean'


class AggregationFunction(Enum):
    MEAN = 'mean'
    MAX = 'max'
    MIN = 'min'
    MEDIAN = 'median'
    MEAN_NO_OUTLIERS = 'mean_no_outliers'
    MEDIAN_NO_OUTLIERS = 'median_no_outliers'


class OverflowFix(Enum):
    ENABLE = 'enable'
    DISABLE = 'disable'
    FIRST_LAYER_ONLY = 'first_layer_only'


@dataclass
class WeightsEstimatorParameters():
    statistic_function:Optional[WeightsStatisticFunction] = None 
    statistic_function_params: Optional[Dict] = None
    clipping_value: Optional[float] = None


@dataclass
class ActivationsEstimatorParameters():
    statistic_function: Optional[ActivationsStatisticFunction] = None 
    statistic_function_params: Optional[Dict] = None
    aggregation_function: Optional[AggregationFunction] = None
    clipping_value: Optional[float] = None


@dataclass
class ActivationsQuantizationParameters:
    num_bits: Optional[int] = None
    mode: Optional[QuantizationMode] = None
    per_channel: Optional[bool] = None # granularity
    range_minimum_estimator: Optional[ActivationsEstimatorParameters] = None
    range_maximum_estimator: Optional[ActivationsEstimatorParameters] = None


@dataclass
class WeightsQuantizationParameters:
    num_bits: Optional[int] = None
    mode: Optional[QuantizationMode] = None
    per_channel: Optional[bool] = None 
    #granularity
    half_range: Optional[bool] = None
    narrow_range: Optional[bool] = None
    range_minimum_estimator: Optional[WeightsEstimatorParameters] = None
    range_maximum_estimator: Optional[WeightsEstimatorParameters] = None


@dataclass
class BiasCorrectionParameters:
    apply_for_all_nodes = Optional[bool] = None
    threshold = Optional[float] = None


@dataclass
class AdvancedQuantizationParameters:
    # quantization configurations for activations and weights
    activations: Optional[ActivationsQuantizationParameters] = None
    weights: Optional[WeightsQuantizationParameters] = None
    #activations_quantization_params: Optional[ActivationsQuantizationParameters] = None
    #weights_quantization_params: Optional[WeightsQuantizationParameters] = None
       
    # BiasCorrection algorithm parameters
    bias_correction_params: Optional[BiasCorrectionParameters] = None
    
    # general parameters
    overflow_fix: Optional[OverflowFix] = None
    inplace_statistics: Optional[bool] = None
    
    # backend specific parameters
    backend_params: Optional[Dict[str, Any]] = None
     

def quantize(
    model: TModel,
    calibration_dataset: Dataset,
    preset: QuantizationPreset = QuantizationPreset.PERFORMANCE,
    target_device: TargetDevice = TargetDevice.ANY,
    subset_size: int = 300,
    fast_bias_correction: bool = True,
    model_type: Optional[ModelType] = None,
    ignored_scope: Optional[IgnoredScope] = None,
    advanced_parameters: Optional[AdvancedQuantizationParameters()] = None) -> TModel:
pass

# how to use

advanced_parameters = AdvancedQuantizationParameters(
    activations=ActivationsQuantizationParameters(
        num_bits=8, 
        range_minimum_estimator=ActivationRangeEstimatorParameters(
            statistic_function=ActivationStatisticFunction.MIN,
            aggregator=AggregatorFunction.MEAN),
        range_maximum_estimator=ActivationRangeEstimatorParameters(
            statistic_function=ActivationStatisticFunction.MAX,
            aggregator=AggregatorFunction.MEAN)),
    weights=WeightsQuantizationParameters(
        num_bits=8,
        per_channel=True),
    bias_correction_params = BiasCorrectionParameters(apply_for_all_nodes = True)
    backend_params={
        "num_infer_request": 2
    }
)

qmodel = quantize(model, calibration_dataset=dataset, advanced_parameters=advanced_parameters)